![Marcel](pics/Marcel.png) 

#  Segmenting and Clustering Neighborhoods in Toronto 


- __This is assigment from week 3  module 9 for the completion of the IBM Data Science Professional Certificate__

<HR>

> #### <span style="color:green">  <strong> Assignment:
>
>    For this assignment, I will explore and cluster the neighborhoods in Toronto.
>
>    1. I start by creating this Notebook for this assignment.
>    2. I will use this Notebook to build the code to scrape the following Wikipedia page,   
>    [https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M),  
>    in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown >below:
>       ![Neighbourhoods|1000x486,50%](pics/wk3pic.jpg) 
>    
>    3. To create the above dataframe:
>       - The dataframe will consist of three columns: __PostalCode, Borough, and Neighborhood__
>       - Only process the cells that have an assigned borough and ignore cells with a borough that is "Not assigned".
>       - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
>       - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
>       - In the last cell of this notebook, I will use the .shape method to print the number of rows of my dataframe.
>
>    
>    4. I will submit a link to this Notebook on my Github repository. (10 marks)
>
>    5. I will use pandas to scrape the webpage.
><HR>

In [511]:
# Import Libraries to be used
import pandas as pd    # library for data analsysis
import requests        # library to handle requests
import lxml.html as lh #https://lxml.de/lxmlhtml.html

In [512]:
# using pandas to scrape the website
url  = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
tables = pd.read_html(url)
tables[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [513]:
df = tables[0]

# Determine the dimension
df.shape 

(288, 3)

In [514]:
# Determine how many Borough's are 'Not assigned'
print(df['Borough'].value_counts())

Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64


In [515]:
# All entries where 'Borough' == 'Not Assigned' are absent from new df2
df2 = df[df.Borough != 'Not assigned'] 

In [516]:
# Determine new Dimenion where there are no more Borough's with value 'Not assigned'
df2.shape 

(211, 3)

> ##### <strong> So a drop from 288 to 211. 
    - This means 77 record where Borough = 'Not assigned' were eliminated 

In [517]:
#A check to be sure
print(df2['Borough'].value_counts())

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64


> ##### <strong> Now we look if there are remaining Neigbourhoods that are 'Not assigned'

In [518]:
count_Not_Assigned = df2['Neighbourhood'].str.contains('Not assigned').sum()
if count_Not_Assigned > 0:
    print ("There are {m} Neighbourhoods 'Not_Assigned' ".format(m = count_Not_Assigned))
else: print ("All is good")

There are 1 Neighbourhoods 'Not_Assigned' 


In [519]:
# Find the records where Neighbourhood == 'Not assigned'
df2.loc[df2.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


> ##### <strong> Now replace values of 'Not assigned'. Checking total dataframe in case there had been more then 1

In [523]:
for row_index,row in df2.iterrows():
    if df2['Neighbourhood'][row_index]=='Not assigned':
        df2['Neighbourhood'][row_index] = df2['Borough'][row_index]
    else:
        continue

In [525]:
df2.loc[df2.Postcode == 'M7A']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


> ### So the change was succesful
 - Remains to group the neighbourhoods per Postcode

In [526]:
df_schoon = df2.reset_index(drop=True)
print(df_schoon.shape)
df_schoon.sort_values(by='Postcode').head(6)

(211, 3)


,Postcode,Borough,Neighbourhood
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern
23,M1C,Scarborough,Port Union
22,M1C,Scarborough,Rouge Hill
21,M1C,Scarborough,Highland Creek
33,M1E,Scarborough,Guildwood


> #### Now groupby Postcode and Bouorough 
   - to see same postcalcodes with different neighbourhoods.
   - to use apply to join neighbourhoods together when the have the same postalcode and place result back into a datframe

In [527]:
df_finish = df_schoon.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).to_frame(name = 'Neighbourhood').reset_index()

In [528]:
df_finish.to_csv('Tor_Post.csv', sep=';')
df_finish.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [529]:
df_finish.shape

(103, 3)

> #### <strong> So there are 103 records with unique postal codes

> ##### GitHub-link :  
[Link to Notebook : Capstone wk3 - IBM Data Science Clustering Neighbourhoods.ipynb on GitHub](https://github.com/MarcelVos1966/Capstone-project/blob/master/Capstone%20wk3%20-%20IBM%20Data%20Science%20Clustering%20Neighbourhoods.ipynb "Link to Notebook on GitHub")

> #### I wil use a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data
> - Because of the instability to get the geographical coordinates of the neighborhoods using the Geocoder package <br>
> It wil produce the following dataframe.<br>
> ![Geo Coordinates|708x381](pics/wk3geo.jpg) 
<HR>

In [532]:
# Import csv_file with coordinates
df_geo = pd.read_csv('GeoData.csv', sep=';')

In [533]:
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,438.066.863,-791.943.534
1,M1C,437.845.351,-791.604.971
2,M1E,437.635.726,-791.887.115
3,M1G,437.709.921,-792.169.174
4,M1H,43.773.136,-792.394.761


In [534]:
# Determine dimension
df_geo.shape

(103, 3)

> #### Unify in name the columns with the 3 alfanumeric code (potcode and Postal Code)

In [535]:
df_finish.rename(columns={'Postcode': 'Postalcode'}, inplace=True)
df_geo.rename(columns={'Postal Code': 'Postalcode'}, inplace=True)
df_geo.head()

,Postalcode,Latitude,Longitude
0,M1B,438.066.863,-791.943.534
1,M1C,437.845.351,-791.604.971
2,M1E,437.635.726,-791.887.115
3,M1G,437.709.921,-792.169.174
4,M1H,43.773.136,-792.394.761


> #### Merge the dataframes to get the final dataframe with all relevant data combined.
  - merging is done per Postalcode

In [536]:
df_merged_data = pd.merge(df_finish, df_geo, on='Postalcode')

In [537]:
df_merged_data.to_csv('TorGeo.csv', sep=';')

In [538]:
df_merged_data.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",438.066.863,-791.943.534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",437.845.351,-791.604.971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",437.635.726,-791.887.115
3,M1G,Scarborough,Woburn,437.709.921,-792.169.174
4,M1H,Scarborough,Cedarbrae,43.773.136,-792.394.761
5,M1J,Scarborough,Scarborough Village,437.447.342,-792.394.761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",437.279.292,-792.620.294
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",437.111.117,-792.845.772
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716.316,-792.394.761
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692.657,-792.648.481


> # End Notebook
>
> ![Geo Coordinates|708x381](pics/end.png) 

<HR>